# Advanced decorators

This notebook contains a series of samples on how to implement decorators on Python

# Imports

In [ ]:
import functools
from enum import Enum
from typing import Set
import time

# Simple

## Func without parameters

In [ ]:
def simple_decorator(func):
    # It is not required, but it is important to use functools.wraps to 
    # properly preserve information about the original function
    @functools.wraps(func)
    def wrapper():
        print('Before calling func')
        func()
        print('After calling func')
    return wrapper

In [ ]:
@simple_decorator
def func_without_param():
    print(f'Hello World')

In [ ]:
help(func_without_param)

# Without functool.wraps help function returns:
# Help on function wrapper in module __main__:
# wrapper()

# After applying functools.wraps it returned the original function properly:
# Help on function func_without_param in module __main__:
# func_without_param()

In [ ]:
func_without_param()

## Proxying parameters

In [ ]:
def proxy_parameters_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Before calling func')
        func(*args, **kwargs)
        print('After calling func')
    return wrapper

In [ ]:
@proxy_parameters_decorator
def some_func(name:str, age: int):
    print(f'Hi {name}. Your are {age} years old.')

In [ ]:
some_func('Alisson', 35)

## Real cases

### Timing functions

In [ ]:
def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):

        start_time = time.perf_counter()

        fn_result = func(*args, **kwargs)

        end_time = time.perf_counter()

        run_time = end_time - start_time
        print(f'Function {func.__name__} executed in {run_time:.4f} sec')

        return fn_result

    return wrapper

In [ ]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([number**2 for number in range(10_000)])

In [ ]:
# Function will be executed and the elapsed time will be printer
waste_some_time(100)

# Advanced

## Decorator with params

In [ ]:
# Explanation
# When whe use @ Python expects a function that can receive an another function as it's 
# first parameter.Since we are trying to use a function that receiveis it's own
# parameters, we have to make this function to return an another function that
# meets Python requirement.

# Here we define a function that receveis it's own parameters
def decorator_with_param(decor_param: str):

    # This is the function that will be called by Python receiving the
    # decorated function as it's parameter
    def decorator(func):
        
        # The rest of the code is the same of a decorator without params
        # the only difference is that now we can access the decorator paramters

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print (f'The parameter passed to the decorator was {decor_param}')
            return func(*args, **kwargs)

        return wrapper
    return decorator

In [ ]:
@decorator_with_param(decor_param=123)
def fn_decorated_with_params(fn_param):
    print(f'Param sent to the decorated function was {fn_param}')

In [ ]:
fn_decorated_with_params('Alisson Lima')

## Decorator with optional params

In [ ]:
# Explanation
# Since the function to decorate is only passed in directly if the decorator 
# is called without arguments, the function must be an optional argument. 
# This means that the decorator arguments must all be specified by keyword. 
# We can enforce this with the special asterisk (*) syntax, which means that 
# all the following parameters are keyword-only:
# Source: https://realpython.com/primer-on-python-decorators/#inner-functions

# Here we define a function that receveis it's own parameters
def decorator_with_optional_param(_func=None, *, decor_param: str = 123):

    # This is the function that will be called by Python receiving the
    # decorated function as it's parameter
    def decorator(func):
        
        # The rest of the code is the same of a decorator without params
        # the only difference is that now we can access the decorator paramters

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print (f'The parameter passed to the decorator was {decor_param}')
            return func(*args, **kwargs)

        return wrapper
    
    # Here is the key difference. We need to validate if the function has or hasn't 
    # received an another function as it's first parameter.    
    
    if _func is None:        
        # If it hasn't, so it was called with parameters and we have to
        # return our decorator function so Python will sent the function
        # to it
        return decorator
    
    else:
        # If it has, so it was called without parameters, so we have
        # to call the decorator function passing the function to it
        return decorator(_func)

In [ ]:
@decorator_with_optional_param
def fn_decorated_with_optional_params_absent(fn_param):
    print(f'Param sent to the decorated function was {fn_param}')

fn_decorated_with_optional_params_absent('Alisson Lima')

In [ ]:
@decorator_with_optional_param(decor_param=456)
def fn_decorated_with_optional_params_present(fn_param):
    print(f'Param sent to the decorated function was {fn_param}')

fn_decorated_with_optional_params_present('Alisson Lima')

## Real cases

### Authorizing access

In [ ]:
class AccessLevel(Enum):
    ADMIN = 1
    USER = 2

In [ ]:
def authorize(required_levels: Set[AccessLevel]):

    def decorator(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):            
            user_lvl = kwargs['access_level']
            if user_lvl not in required_levels:
                raise Exception('Access Not Allowed') 
            
            return func(*args, **kwargs)
            
        return wrapper
    
    return decorator


In [ ]:
@authorize([AccessLevel.ADMIN])
def get_personal_data(username: str, access_level: int):
    return f'Hi {username}. You have access personal data'


In [ ]:
@authorize([AccessLevel.ADMIN, AccessLevel.USER])
def get_public_data(username: str, access_level: int):
    return f'Hi {username}. You have access personal data'


In [ ]:
# It will work since Admins can access personal data
get_personal_data(username='Alisson', access_level=AccessLevel.ADMIN)

In [ ]:
# It will throw an Exception since the USER level cannot
# acces the get_personal_data function
get_personal_data(username='John Doe', access_level=AccessLevel.USER)

In [ ]:
# Both of this will work just fine since everybody can
# access public data
print(get_public_data(username='Alisson', access_level=AccessLevel.ADMIN))
print(get_public_data(username='John Doe', access_level=AccessLevel.USER))